# Import libraries

In [30]:
import pandas as pd
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller

from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel

import pandas_datareader as dr
import plotly.express as px


from pandas.plotting import scatter_matrix


import warnings
warnings.filterwarnings('ignore')

# Download data

In [31]:
df = pd.read_csv('russian_data_without_prefs.csv')

# drop zero returns (weekends and holidays)
df.drop(df[df.filter(regex='return_').sum(axis = 1) == 0].index, axis=0, inplace=True)

# Construct classical factors SMB, HML and MOM

In [32]:
data_mc = df.filter(regex='CURRENT_MARKET_CAP')
data_mb = df.filter(regex='PX_TO_BOOK')
data_ret = df.filter(regex='return')
data_mb = 1/data_mb

data_mc_1 = data_mc.loc[:,'CURRENT_MARKET_CAP_SHARE_CLASS_SBER_RM_Equity':]
data_ret_1 = data_ret.loc[:,'return_SBER_RM_Equity':]
data_mb_1 = data_mb.loc[:,'PX_TO_BOOK_RATIO_SBER_RM_Equity':]

# в них слишком много нанов
data_mc = data_mc_1.drop(['CURRENT_MARKET_CAP_SHARE_CLASS_MAIL_RM_Equity',
                          'CURRENT_MARKET_CAP_SHARE_CLASS_POGR_RM_Equity',
                          'CURRENT_MARKET_CAP_SHARE_CLASS_TCSG_RM_Equity',
                          'CURRENT_MARKET_CAP_SHARE_CLASS_FIVE_RM_Equity',
                          'CURRENT_MARKET_CAP_SHARE_CLASS_DSKY_RM_Equity',
                          'CURRENT_MARKET_CAP_SHARE_CLASS_RNFT_RM_Equity',
                          'CURRENT_MARKET_CAP_SHARE_CLASS_SFIN_RM_Equity',
                          'CURRENT_MARKET_CAP_SHARE_CLASS_CBOM_RM_Equity',
                          'CURRENT_MARKET_CAP_SHARE_CLASS_UWGN_RM_Equity',
                          'CURRENT_MARKET_CAP_SHARE_CLASS_RUAL_RM_Equity',
                          'CURRENT_MARKET_CAP_SHARE_CLASS_AGRO_RM_Equity',
                          'CURRENT_MARKET_CAP_SHARE_CLASS_YNDX_RM_Equity',
                          'CURRENT_MARKET_CAP_SHARE_CLASS_POLY_RM_Equity',
                          'CURRENT_MARKET_CAP_SHARE_CLASS_QIWI_RM_Equity'], axis=1)

data_ret = data_ret_1.drop(['return_MAIL_RM_Equity','return_POGR_RM_Equity','return_TCSG_RM_Equity','return_FIVE_RM_Equity',
                            'return_DSKY_RM_Equity','return_RNFT_RM_Equity', 'return_SFIN_RM_Equity', 'return_CBOM_RM_Equity',
                            'return_UWGN_RM_Equity','return_RUAL_RM_Equity','return_AGRO_RM_Equity', 'return_YNDX_RM_Equity',
                            'return_POLY_RM_Equity', 'return_QIWI_RM_Equity'], axis=1)

data_mb = data_mb_1.drop(['PX_TO_BOOK_RATIO_MAIL_RM_Equity','PX_TO_BOOK_RATIO_POGR_RM_Equity','PX_TO_BOOK_RATIO_TCSG_RM_Equity','PX_TO_BOOK_RATIO_FIVE_RM_Equity',
                          'PX_TO_BOOK_RATIO_DSKY_RM_Equity','PX_TO_BOOK_RATIO_RNFT_RM_Equity', 'PX_TO_BOOK_RATIO_SFIN_RM_Equity', 'PX_TO_BOOK_RATIO_CBOM_RM_Equity',
                          'PX_TO_BOOK_RATIO_UWGN_RM_Equity','PX_TO_BOOK_RATIO_RUAL_RM_Equity','PX_TO_BOOK_RATIO_AGRO_RM_Equity', 'PX_TO_BOOK_RATIO_YNDX_RM_Equity',
                          'PX_TO_BOOK_RATIO_POLY_RM_Equity','PX_TO_BOOK_RATIO_QIWI_RM_Equity'], axis=1)

Date = pd.DataFrame(df['Date'])

data_mc = pd.merge(Date, data_mc, left_index=True, right_index=True)
data_ret = pd.merge(Date, data_ret, left_index=True, right_index=True)
data_mb = pd.merge(Date, data_mb, left_index=True, right_index=True)

In [ ]:
# Гиперпараметры моделей
# для квантилей M/B ratio
quant_hml_hp = [0.3, 0.7, 1]
# для квантилей Market Capitalization
quant_smb_hp = [0.5, 1]
# для квантилей дневных доходностей
quant_ret_hp = [0.5 ,1]

# Создаём пустые списки, куда будем заносить значения факторов из моделей Фамы-Френча
s_l_list = []
s_m_list = []
s_h_list = []

b_l_list = []
b_m_list = []
b_h_list = []

high_ret_list = []
low_ret_list = []
    

# Переименуем одинаково все столбцы для показателей доходности, M/B ratio, Market Capitalization для каждой компании
# Это производится с целью последующей обработки данных где разные показатели в 3 таблицах имеют одинаковые имена столбцов,
# стобы по ним можно было обращаться в рамках таблиц (выше было проверено соответствие для каждой компании, что 
# все показатели для каждой компании находятся в правильном порядке как между собой так и между компаний)
data_ret.columns = [i for i in range(len(data_ret.columns))]
data_mb.columns = [i for i in range(len(data_mb.columns))]
data_mc.columns = [i for i in range(len(data_mc.columns))]
    
# переберём все строки в датафрейме и для каждой сделаем следующую процедуру    
for i in range(len(data_ret)):
    
    print('Date is {}'.format(data_ret.iloc[i,0]))
    # делаем лист из 3 значений m/b ratio которые соотвтствуют границам квантилей
    quants_hml = []
    for quant_hml in [quant_hml_hp[0], quant_hml_hp[1], quant_hml_hp[2]]:
        curr_q = data_mb.iloc[i, 1:].quantile(quant_hml)
        quants_hml.append(curr_q)
    
    print('{} quantiles for Market to Book Ratioe are {}'.format(quant_hml_hp, quants_hml))
    # делаем лист из 2 значений market cap которые соотвтствуют границам квантилей
    quants_smb = []    
    for quant_smb in [quant_smb_hp[0], quant_smb_hp[1]]:
        curr_quant = data_mc.iloc[i, 1:].quantile(quant_smb)
        quants_smb.append(curr_quant)
    
    print('{} quantiles for Market Capitalization are {}'.format(quant_smb_hp, quants_smb))
    # делаем лист из 2 значений доходностей которые соотвтствуют границам квантилей
    quants_ret = []
    for quant_ret in [quant_ret_hp[0], quant_ret_hp[1]]:
        curr_q_ret = data_ret.iloc[i, 1:].quantile(quant_ret)
        quants_ret.append(curr_q_ret)
    
    print('{} quantiles for daily returns are {}'.format(quant_ret_hp, quants_ret))
    
    # выбираем в три листа индексы колонок датафрейма, содержащего m/b ratio, которые соответствуют 
    # компаниям по условиям попадающим в соответствующий квантиль, далее по этому индексу мы будем выбирать 
    # доходность в датафрейме доходностей, я специально подбирал так, чтобы строка и столбец в разных датафреймах 
    # относились к одной и той же дате и компании но содержали разные показатели
    columns_low = pd.DataFrame(data_mb.iloc[i, 1:][data_mb.iloc[i, 1:] <= quants_hml[0]]).T.columns
    columns_mean = pd.DataFrame(data_mb.iloc[i, 1:][(data_mb.iloc[i, 1:] > quants_hml[0]) & (data_mb.iloc[i, 1:] <= quants_hml[1])]).T.columns
    columns_high = pd.DataFrame(data_mb.iloc[i, 1:][(data_mb.iloc[i, 1:] > quants_hml[1]) & (data_mb.iloc[i, 1:] <= quants_hml[2])]).T.columns
    
    columns_small = pd.DataFrame(data_mc.iloc[i, 1:][data_mc.iloc[i, 1:] <= quants_smb[0]]).T.columns
    columns_big = pd.DataFrame(data_mc.iloc[i, 1:][(data_mc.iloc[i, 1:] > quants_smb[0]) & (data_mc.iloc[i, 1:] <= quants_smb[1])]).T.columns
    
    columns_low_ret = pd.DataFrame(data_ret.iloc[i, 1:][data_ret.iloc[i, 1:] <= quants_ret[0]]).T.columns
    columns_high_ret = pd.DataFrame(data_ret.iloc[i, 1:][(data_ret.iloc[i, 1:] > quants_ret[0]) & (data_ret.iloc[i, 1:] <= quants_ret[1])]).T.columns
    
    # расчитываем средние по группам доходностии для дальнейшего использования в формировании факторов
    # Взвешиваем доходность по капитализации компаний, удовлетворяющих соответстствующим условиям s_*
    s_l = np.average(data_ret.iloc[i, list(set(columns_low).intersection(set(columns_small)))], weights=data_mc.iloc[i,data_ret.iloc[i, list(set(columns_low).intersection(set(columns_small)))].index])
    s_m = np.average(data_ret.iloc[i, list(set(columns_mean).intersection(set(columns_small)))], weights=data_mc.iloc[i,data_ret.iloc[i, list(set(columns_mean).intersection(set(columns_small)))].index]) 
    s_h = np.average(data_ret.iloc[i, list(set(columns_high).intersection(set(columns_small)))], weights=data_mc.iloc[i,data_ret.iloc[i, list(set(columns_high).intersection(set(columns_small)))].index])
    
    print('s_l = {}, s_m = {}, s_h = {}'.format(s_l, s_m, s_h))
    
    # Взвешиваем доходность по капитализации компаний, удовлетворяющих соответстствующим условиям b_*
    b_l = np.average(data_ret.iloc[i, list(set(columns_low).intersection(set(columns_big)))], weights=data_mc.iloc[i,data_ret.iloc[i, list(set(columns_low).intersection(set(columns_big)))].index])
    b_m = np.average(data_ret.iloc[i, list(set(columns_mean).intersection(set(columns_big)))], weights=data_mc.iloc[i,data_ret.iloc[i, list(set(columns_mean).intersection(set(columns_big)))].index]) 
    b_h = np.average(data_ret.iloc[i, list(set(columns_high).intersection(set(columns_big)))], weights=data_mc.iloc[i,data_ret.iloc[i, list(set(columns_high).intersection(set(columns_big)))].index])
    
    
    print('b_l = {}, b_m = {}, b_h = {}'.format(b_l, b_m, b_h))
    
    # Взвешиваем высокодоходные и низкодоходные компании
    high_ret = np.average(data_ret.iloc[i, columns_high_ret], weights=data_mc.iloc[i, columns_high_ret])
    low_ret = np.average(data_ret.iloc[i, columns_low_ret], weights=data_mc.iloc[i, columns_low_ret])
    
    print('high_ret = {}, low_ret = {}'.format(high_ret, low_ret))
    
    # заполняем лист из факторов, каждое значение для соответствующей даты по порядку
    s_l_list.append(s_l)
    s_m_list.append(s_m)
    s_h_list.append(s_h)

    b_l_list.append(b_l)
    b_m_list.append(b_m)
    b_h_list.append(b_h)
    
    high_ret_list.append(high_ret)
    low_ret_list.append(low_ret)
    
    print("##################################################")


    
FF_count = pd.DataFrame({'S/L':s_l_list, 'S/M':s_m_list, 'S/H':s_h_list, 
                         'B/L':b_l_list, 'B/M':b_m_list, 'B/H':b_h_list, 
                         'high_ret':high_ret_list, 'low_ret':low_ret_list})

FF_count['SMB'] = (FF_count['S/L']+FF_count['S/M']+FF_count['S/H'])/3 - (FF_count['B/L']+FF_count['B/M']+FF_count['B/H'])/3
FF_count['HML'] = (FF_count['S/L']+FF_count['B/L'])/2 - (FF_count['S/H']+FF_count['B/H'])/2
FF_count['MOM'] = FF_count['high_ret'] - FF_count['low_ret']
#FF_count.to_csv('FF_count.csv')

# Read the data we made

In [33]:
FF_count = pd.read_csv('FF_count.csv')

In [34]:
data_mc = df.filter(regex='CURRENT_MARKET_CAP')
data_mb = df.filter(regex='PX_TO_BOOK')
data_ret = df.filter(regex='return')
data_mb = 1/data_mb

# в них слишком много нанов
data_mc = data_mc.drop(['CURRENT_MARKET_CAP_SHARE_CLASS_MAIL_RM_Equity',
                          'CURRENT_MARKET_CAP_SHARE_CLASS_POGR_RM_Equity',
                          'CURRENT_MARKET_CAP_SHARE_CLASS_TCSG_RM_Equity',
                          'CURRENT_MARKET_CAP_SHARE_CLASS_FIVE_RM_Equity',
                          'CURRENT_MARKET_CAP_SHARE_CLASS_DSKY_RM_Equity',
                          'CURRENT_MARKET_CAP_SHARE_CLASS_RNFT_RM_Equity',
                          'CURRENT_MARKET_CAP_SHARE_CLASS_SFIN_RM_Equity',
                          'CURRENT_MARKET_CAP_SHARE_CLASS_CBOM_RM_Equity',
                          'CURRENT_MARKET_CAP_SHARE_CLASS_UWGN_RM_Equity',
                          'CURRENT_MARKET_CAP_SHARE_CLASS_RUAL_RM_Equity',
                          'CURRENT_MARKET_CAP_SHARE_CLASS_AGRO_RM_Equity',
                          'CURRENT_MARKET_CAP_SHARE_CLASS_YNDX_RM_Equity',
                          'CURRENT_MARKET_CAP_SHARE_CLASS_POLY_RM_Equity',
                          'CURRENT_MARKET_CAP_SHARE_CLASS_QIWI_RM_Equity'], axis=1)

data_ret = data_ret.drop(['return_MAIL_RM_Equity','return_POGR_RM_Equity','return_TCSG_RM_Equity','return_FIVE_RM_Equity',
                            'return_DSKY_RM_Equity','return_RNFT_RM_Equity', 'return_SFIN_RM_Equity', 'return_CBOM_RM_Equity',
                            'return_UWGN_RM_Equity','return_RUAL_RM_Equity','return_AGRO_RM_Equity', 'return_YNDX_RM_Equity',
                          'return_POLY_RM_Equity', 'return_QIWI_RM_Equity'], axis=1)

data_mb = data_mb.drop(['PX_TO_BOOK_RATIO_MAIL_RM_Equity','PX_TO_BOOK_RATIO_POGR_RM_Equity','PX_TO_BOOK_RATIO_TCSG_RM_Equity','PX_TO_BOOK_RATIO_FIVE_RM_Equity',
                            'PX_TO_BOOK_RATIO_DSKY_RM_Equity','PX_TO_BOOK_RATIO_RNFT_RM_Equity', 'PX_TO_BOOK_RATIO_SFIN_RM_Equity', 'PX_TO_BOOK_RATIO_CBOM_RM_Equity',
                            'PX_TO_BOOK_RATIO_UWGN_RM_Equity','PX_TO_BOOK_RATIO_RUAL_RM_Equity','PX_TO_BOOK_RATIO_AGRO_RM_Equity', 'PX_TO_BOOK_RATIO_YNDX_RM_Equity',
                        'PX_TO_BOOK_RATIO_POLY_RM_Equity', 'PX_TO_BOOK_RATIO_QIWI_RM_Equity'], axis=1)

Date = pd.DataFrame(df['Date'])

data_mc = pd.merge(Date, data_mc, left_index=True, right_index=True)
data_ret = pd.merge(Date, data_ret, left_index=True, right_index=True)
data_mb = pd.merge(Date, data_mb, left_index=True, right_index=True)

returns = data_ret.iloc[1:,:]
returns.index = returns.index - 1

data = pd.merge(returns, FF_count[['SMB','HML', 'MOM']], left_index=True, right_index=True)

# Using R.Hamada formula let's make corrections coeficients we will receive before Market risk premium, SMB, HML

In [35]:
data.set_index('Date', inplace=True)
data_debt_to_mkt_cap = df.filter(regex = 'DEBT_TO_MKT_CAP')
data_debt_to_mkt_cap = pd.merge(Date, data_debt_to_mkt_cap, left_index=True, right_index=True)
data_debt_to_mkt_cap.set_index('Date', inplace=True)


data_debt_to_mkt_cap_clear = data_debt_to_mkt_cap.loc[data.index[0]:data.index[-1],:].iloc[:,1:].drop(['DEBT_TO_MKT_CAP_MAIL_RM_Equity',
                                                                                                 'DEBT_TO_MKT_CAP_POGR_RM_Equity',
                                                                                                 'DEBT_TO_MKT_CAP_TCSG_RM_Equity',
                                                                                                 'DEBT_TO_MKT_CAP_FIVE_RM_Equity',
                                                                                                 'DEBT_TO_MKT_CAP_DSKY_RM_Equity',
                                                                                                 'DEBT_TO_MKT_CAP_RNFT_RM_Equity',
                                                                                                 'DEBT_TO_MKT_CAP_SFIN_RM_Equity',
                                                                                                 'DEBT_TO_MKT_CAP_CBOM_RM_Equity',
                                                                                                 'DEBT_TO_MKT_CAP_UWGN_RM_Equity',
                                                                                                 'DEBT_TO_MKT_CAP_RUAL_RM_Equity',
                                                                                                 'DEBT_TO_MKT_CAP_AGRO_RM_Equity',
                                                                                                 'DEBT_TO_MKT_CAP_YNDX_RM_Equity'], axis=1)

# We work with expectations on the market that's why it in nesessary to make 1 month lag (30 days) for debt to market cap ratio
window = 30
data_debt_to_mkt_cap_clear = data_debt_to_mkt_cap_clear.shift(window).iloc[window:,:]
data = data.iloc[window:,:]
# t - tax rate
# according to Russian rules it equals 20%
t = 0.2
#SMB
for d_mcap_ratio in data_debt_to_mkt_cap_clear.columns:
    data['Hamada_d_mcap_ratio_SMB_for_{}'.format(d_mcap_ratio[16:20])] = pd.DataFrame(data['SMB']) * (1 + (1-t)*data_debt_to_mkt_cap_clear.filter(regex = d_mcap_ratio[16:20], axis = 1).values)
  
#HML
for d_mcap_ratio in data_debt_to_mkt_cap_clear.columns:
    data['Hamada_d_mcap_ratio_HML_for_{}'.format(d_mcap_ratio[16:20])] = pd.DataFrame(data['HML']) * (1 + (1-t)*data_debt_to_mkt_cap_clear.filter(regex = d_mcap_ratio[16:20], axis = 1).values)


data.drop(columns = ['Hamada_d_mcap_ratio_SMB_for_QIWI',
                     'Hamada_d_mcap_ratio_HML_for_QIWI',
                     'Hamada_d_mcap_ratio_SMB_for_POLY',
                     'Hamada_d_mcap_ratio_HML_for_POLY'], inplace=True)
data

,return_IMOEX_Index,return_SBER_RM_Equity,return_GAZP_RM_Equity,return_LKOH_RM_Equity,return_GMKN_RM_Equity,return_NVTK_RM_Equity,return_SNGS_RM_Equity,return_PLZL_RM_Equity,return_TATN_RM_Equity,return_ROSN_RM_Equity,...,Hamada_d_mcap_ratio_HML_for_NKNC,Hamada_d_mcap_ratio_HML_for_GCHE,Hamada_d_mcap_ratio_HML_for_SVAV,Hamada_d_mcap_ratio_HML_for_MSTT,Hamada_d_mcap_ratio_HML_for_BSPB,Hamada_d_mcap_ratio_HML_for_MSRS,Hamada_d_mcap_ratio_HML_for_KMAZ,Hamada_d_mcap_ratio_HML_for_RASP,Hamada_d_mcap_ratio_HML_for_OGKB,Hamada_d_mcap_ratio_HML_for_VZRZ
Date,,,,,,,,,,,,,,,,,,,,,
2010-03-29,0.087741,0.062558,0.000912,0.080427,0.182671,0.111388,0.180072,0.016429,0.043141,0.038145,...,NaN,NaN,-0.016485,NaN,-0.015633,NaN,-0.010690,-0.008225,NaN,-0.010690
2010-03-30,0.058706,0.023301,-0.009586,0.052201,0.152624,0.076228,0.152811,-0.012535,0.036806,0.007637,...,NaN,NaN,0.021150,NaN,0.020774,NaN,0.014245,0.010976,NaN,0.014222
2010-03-31,0.068537,0.038480,0.001922,0.063672,0.166173,0.081260,0.157183,-0.004964,0.036799,0.008667,...,NaN,NaN,-0.043453,NaN,-0.043933,NaN,-0.030425,-0.023460,NaN,-0.030485
2010-04-01,0.100061,0.093875,0.039632,0.080956,0.200051,0.095372,0.180973,0.002818,0.055191,0.033242,...,NaN,NaN,-0.003021,NaN,-0.002979,NaN,-0.002051,-0.001584,NaN,-0.002056
2010-04-02,0.089278,0.105903,0.023143,0.069109,0.177871,0.085347,0.149127,-0.001707,0.029562,-0.001481,...,NaN,NaN,0.038673,NaN,0.037827,NaN,0.025963,0.020135,NaN,0.025770
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-10-06,-0.045958,-0.076986,-0.083382,-0.144096,-0.054297,-0.027438,-0.038448,-0.092224,-0.189351,-0.006641,...,-0.026781,-0.031899,-0.030673,-0.044401,-0.124495,-0.044266,-0.055047,-0.019189,-0.025754,-0.042644
2020-10-07,-0.071075,-0.078507,-0.108340,-0.165264,-0.068471,-0.036498,-0.087389,-0.101977,-0.199757,-0.021372,...,-0.005530,-0.006562,-0.006324,-0.009160,-0.025118,-0.009112,-0.011438,-0.003959,-0.005315,-0.008798
2020-10-08,-0.056556,-0.084158,-0.097518,-0.140013,-0.038664,-0.017992,-0.068766,-0.073465,-0.189472,0.007288,...,-0.018190,-0.021593,-0.020811,-0.030191,-0.083071,-0.030210,-0.037112,-0.013029,-0.017542,-0.028954


# Add G-curve as risk free rate

In [36]:
g_curve = pd.read_excel('G-curve_ON.xlsx')
g_curve.rename(columns = {'g_ON':'risk_free_rate_based_on_g_curve'}, inplace=True)
g_curve['Date'] = pd.to_datetime(g_curve['Date'])
date = []
for i in range(len(g_curve)):
    date.append(str(g_curve['Date'][i])[0:10])
  
g_curve['Date'] = date
g_curve.set_index('Date', inplace=True)
daily_risk_free_rate = g_curve/12
#data.set_index('Date', inplace=True)

data = pd.merge(data, daily_risk_free_rate.loc['2010-01-19':'2020-06-04',:], left_index = True, right_index=True)
# data.drop(columns=['rf_x', 'rf_y'], inplace=True)

data.reset_index(inplace=True)
data['Market_risk_premium'] = data['return_IMOEX_Index'] - data['risk_free_rate_based_on_g_curve']

In [37]:
data_for_plot = data.reset_index()[['Date', 'risk_free_rate_based_on_g_curve']]
data_for_plot.rename(columns = {'Date':'Год', 'risk_free_rate_based_on_g_curve':'Безрисковая месячная доходность'}, inplace = True)

## Plot risk monthly risk free rate

In [38]:
fig = px.line(data_for_plot, x='Год', y="Безрисковая месячная доходность")
fig.show()

# Add market risk premium

In [39]:
data_for_plot = data[['Date', 'Market_risk_premium']]
data_for_plot.rename(columns = {'Date':'Год', 'Market_risk_premium':'Рыночная премия за риск'}, inplace = True)

## Plot market risk premium

In [40]:
fig = px.line(data_for_plot, x='Год', y="Рыночная премия за риск")
fig.show()

# Add Hamada correction for company debt for market risk premium

In [41]:
#Market risk premium
for d_mcap_ratio in data_debt_to_mkt_cap_clear.columns:
    data['Hamada_beta_MRP_for_{}'.format(d_mcap_ratio[16:20])] = (pd.DataFrame(data['return_IMOEX_Index'] - data['risk_free_rate_based_on_g_curve'])) * (1 + (1-t)*data_debt_to_mkt_cap_clear.loc[data['Date'],:].filter(regex = d_mcap_ratio[16:20], axis = 1).values)



# Add two additional momemtum factors (average over 3 and 7 days)

In [42]:
# тут питон ругается но дело делает, всё нормально
for i in range(0,data.shape[0]-2):
    data.loc[data.index[i+2],'MOM_3'] = ((data.loc[data.index[i],'MOM']+ data.loc[data.index[i+1],'MOM'] + data.loc[data.index[i+2],'MOM'])/3)
    
for i in range(0,data.shape[0]-6):
    data.loc[data.index[i+6],'MOM_7'] = ((data.loc[data.index[i],'MOM']+ data.loc[data.index[i+1],'MOM'] + data.loc[data.index[i+2],'MOM']+
                                          data.loc[data.index[i+3],'MOM']+ data.loc[data.index[i+4],'MOM'] + data.loc[data.index[i+5],'MOM']+
                                          data.loc[data.index[i+6],'MOM'])/7)    

data = data.shift(-6)
data = data.iloc[:-6,:]
data.set_index('Date', inplace=True)    

In [43]:
# Saving data
# data.to_csv('final_data_without_prefs_and_with_HAMADA_factors.csv')
# data_ret.to_csv('data_ret.csv')

# Check the stationarity of received time series (to prevent false regression)

In [44]:
print("p-value if ADF test is {} That's why SMB data is stationare".format(np.round(adfuller(np.array(data['SMB'].dropna()), autolag='AIC',regression = 'c')[1], 30)))
print("p-value if ADF test is {} That's why HML data is stationare".format(adfuller(np.array(data['HML'].dropna()), autolag='AIC',regression = 'c')[1]))
print("p-value if ADF test is {} That's why Market_risk_premium data is stationare".format(adfuller(np.array(data['Market_risk_premium'].dropna()), autolag='AIC',regression = 'c')[1]))
print("p-value if ADF test is {} That's why MOM data is stationare".format(adfuller(np.array(data['MOM'].dropna()), autolag='AIC',regression = 'c')[1]))
print("p-value if ADF test is {} That's why MOM_3 data is stationare".format(adfuller(np.array(data['MOM_3'].dropna()), autolag='AIC',regression = 'c')[1]))
print("p-value if ADF test is {} That's why MOM_7 data is stationare".format(adfuller(np.array(data['MOM_7'].dropna()), autolag='AIC',regression = 'c')[1]))


p-value if ADF test is 2.2919574273281916e-13 That's why SMB data is stationare
p-value if ADF test is 8.155779676004965e-12 That's why HML data is stationare
p-value if ADF test is 1.662444931631533e-18 That's why Market_risk_premium data is stationare
p-value if ADF test is 5.575353483573121e-12 That's why MOM data is stationare
p-value if ADF test is 1.62472339773025e-08 That's why MOM_3 data is stationare
p-value if ADF test is 5.104427753551668e-09 That's why MOM_7 data is stationare


# Download oil and dollar/rubble currency rate as specific for russian stock market factors

In [45]:
oil = pd.read_excel('oil.xls', sheet_name = 'Data 1')
oil = oil.iloc[2:,:2].rename(columns = {'Back to Contents':'Date',
                                        'Data 1: Crude Oil':'crude_oil_wti_price'})
oil.set_index('Date', inplace = True)

dates = []
for date in oil.index:
    dates.append(str(date)[:10])

oil.reset_index(inplace=True)    
    
oil['Date'] = dates
oil.set_index('Date', inplace = True)

x = oil['crude_oil_wti_price']/oil['crude_oil_wti_price'].shift(30)
oil['wti_30d_return'] = np.log(x.astype(np.float))

fig = px.line(oil.reset_index()[['Date', 'crude_oil_wti_price']].rename(columns={'Date':'Год',
                                                                                 'crude_oil_wti_price':'Цена на нефть марки WTI,$'}), x='Год', y="Цена на нефть марки WTI,$")
fig.show()

In [47]:
rubdol = pd.read_csv('rubdol.csv', sep=';')
rubdol = rubdol[::-1]
rubdol.rename(columns = {'#Date':'Date', 'Value 18:50 MSK':'dollar'}, inplace = True)
rubdol.set_index('Date', inplace = True)
rubdol['dollar_30d_return'] = np.log(rubdol['dollar']/rubdol['dollar'].shift(30))

macro = pd.merge(rubdol, oil, left_index=True, right_index=True)
macro = macro.dropna()
macro_data = macro[['dollar_30d_return', 'wti_30d_return']]
